In [1]:
! pip install transformers

In [2]:


import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline

2022-05-29 20:41:24.964231: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
classifier = pipeline("zero-shot-classification", device=0)

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [4]:
#classifier = pipeline("zero-shot-classification", device=0)

### Get the Text Data

In [63]:
headlines = pd.read_csv('/kaggle/input/india-headlines-news-dataset/india-news-headlines.csv')

In [64]:
df_head = headlines.head(100)

In [53]:
samples = headlines[1:100]

In [54]:
strings = samples.headline_text.values

In [55]:
sequence = list(strings)

In [9]:
print(sequence)

["Ayodhya central to BJP's rise & fall at Centre; UP", 'Power situation in Punjab; Haryana critical: Centre', '85 tiny tots take part in baby show', 'Sushma Swaraj directs Indian embassy in Riyadh to rescue Nagda woman held captive', 'AIIMS paper leak: 2 city doctors nabbed', 'HC pulls up officials on hawkers issue', 'Bon voyage: Soon; lighthouses to give directions; weather updates', 'Actor on film shoot in Bhopal harassed by hotel boy at night', 'The right mix of glamour & fun', 'Many schools remain open on Bakr Eid', 'Jumbo rampage hurts two in West Singhbhum', 'Traffic drive nets 3;000 motorists', 'Kishan Lal (ginni)', "IFS officer's PIL: Court prevents PCCF from 'taking any big decision'", 'Vax cash burden may prevent fin aid to needy; fiscal relief to industry', 'wrong side driving', "Kohli; Watson pick Agassi's autobiography as favourite book", "No coaches for India's synchronized swimmers but there's YouTube!", 'Collector asks NDA to check rules on houses near airfield', 'Goa s

# Classification 

In [10]:
candidate_labels = ["politics", "finance", "sports",'entertainment','agriculture','technology']


In [35]:
df_head.head()

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic


,headline_text,label,score
0,Status quo will not be disturbed at Ayodhya; s...,politics,0.239410
1,Fissures in Hurriyat over Pak visit,politics,0.658022
2,America's unwanted heading for India?,politics,0.194707
3,For bigwigs; it is destination Goa,entertainment,0.170343
4,Extra buses to clear tourist traffic,technology,0.184276


In [58]:
output = classifier(sequence, candidate_labels,multi_label = True)

In [59]:
op = pd.DataFrame(output)

In [60]:
op

,sequence,labels,scores
0,Fissures in Hurriyat over Pak visit,"[politics, finance, technology, entertainment,...","[0.6580215096473694, 0.010735863819718361, 0.0..."
1,America's unwanted heading for India?,"[politics, technology, finance, entertainment,...","[0.19470682740211487, 0.06738674640655518, 0.0..."
2,For bigwigs; it is destination Goa,"[entertainment, politics, finance, technology,...","[0.17034222185611725, 0.007796435616910458, 0...."
3,Extra buses to clear tourist traffic,"[technology, entertainment, sports, finance, p...","[0.18427591025829315, 0.05440380796790123, 0.0..."
4,Dilute the power of transfers; says Riberio,"[politics, finance, technology, agriculture, s...","[0.5691625475883484, 0.3738689422607422, 0.360..."
...,...,...,...
94,No report yet on fire in Udyog Bhavan,"[politics, finance, entertainment, technology,...","[0.026448814198374748, 0.02415291965007782, 0...."
95,He is the king of 'five star' industry,"[finance, technology, agriculture, entertainme...","[0.6690065264701843, 0.5558158755302429, 0.132..."
96,Petrol pump sealed for selling adulterated fuel,"[technology, finance, politics, entertainment,...","[0.12637867033481598, 0.013552123680710793, 0...."
97,Milk price up by Rs 1 per litre,"[finance, agriculture, entertainment, technolo...","[0.729042112827301, 0.27204886078834534, 0.064..."


# Sentiment analysis

In [ ]:
sentiment_labels = ['positive','negative']

In [ ]:
sent = classifier(sequence, sentiment_labels)

In [ ]:
sent

### Converting the result into DataFrame with the maximum score

In [65]:
output = []

filter_key1 = ['labels']
filter_key2 = ['scores']

for index, row in df_head.iterrows():
  d = {}
  seq = row['headline_text']
  result = classifier(seq, candidate_labels, multi_label=True)
 # print(result)
  temp_label = list(map(result.get, filter_key1))
  temp_score = list(map(result.get, filter_key2))
 # print(temp)
  d['headline_text'] = row['headline_text']
  d['label'] = temp_label[0][0]
  d['score'] = temp_score[0][0]
  output.append(d) 


In [67]:
 #convert the list of dictionary into pandas DataFrame
 new = pd.DataFrame(output)
 new

,headline_text,label,score
0,Status quo will not be disturbed at Ayodhya; s...,politics,0.239410
1,Fissures in Hurriyat over Pak visit,politics,0.658022
2,America's unwanted heading for India?,politics,0.194707
3,For bigwigs; it is destination Goa,entertainment,0.170343
4,Extra buses to clear tourist traffic,technology,0.184276
...,...,...,...
95,No report yet on fire in Udyog Bhavan,politics,0.026449
96,He is the king of 'five star' industry,finance,0.669008
97,Petrol pump sealed for selling adulterated fuel,technology,0.126379
98,Milk price up by Rs 1 per litre,finance,0.729044
